In [1]:
from pyspark.sql import SparkSession
from src.schema import tbl_schemas
import os
import cml.data_v1 as cmldata

db_name= os.environ["DBNAME"]
tbl_name = []

#db_path =   "s3a://vr-uat1/warehouse/tablespace/external/hive"
#path_prefix = "s3a://vr-uat1/data/bank/"

db_path = os.environ["DBPATH"]
path_prefix = os.environ["PATHPREFIX"]

CONNECTION_NAME = os.environ["CONNECTION_NAME"]
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# spark = (SparkSession
#     .builder
#     .appName("bank-demo")
#     .config("spark.sql.warehouse.dir", db_path)
#     .config("spark.hadoop.fs.s2a.s3guard.ddb.region", "us-east-1")
#     .config("spark.yarn.access.hadoopFileSystems","s3a://vr-uat1/")
#     .master("local[5]") # should be possible to change this to SPARK on Yarn or SPARK on Kubernetes
#     .getOrCreate())

#Create a demo DB if not exists
#spark.sql("".join(["DROP DATABASE IF EXISTS ", db_name," CASCADE"]))
sql_string = "".join(["CREATE DATABASE IF NOT EXISTS ", db_name])
spark.sql(sql_string)

def create_managed_table(schema, path, table_name):

    df = (spark.read.format("csv")
        .option("header", "true")
        .option("schema", "schema")
        .option("nullValue", "NA")  
        .option("timestampFormat", "yyyy-MM-dd'T'HH:mm:SS")
        .option("mode", "failfast")
        .option("path", path)
        .load())
    
    (df.write 
    .mode("overwrite")
#   .option("path", db_path )
    .saveAsTable(db_name + "." + table_name, format="parquet"))
#    spark.sql("REFRESH TABLE " + db_name + "." + table_name).show()
    spark.sql(f"SELECT * from {db_name}.{table_name}").show(3)
    return df

Setting spark.hadoop.yarn.resourcemanager.principal to vishrajagopalan
Hive Session ID = 21cf597c-d779-405e-807b-1ad4d1d77bd1


In [7]:

for index, dict in tbl_schemas.items(): 
    _ =  create_managed_table(tbl_schemas[index]["schema"], path_prefix+ tbl_schemas[index]["path"],  tbl_schemas[index]["table"])
   # db1.show(10)

+--------+----------+------+------+---------+------+---------+--------+----+-------+
|trans_id|account_id|  date|  type|operation|amount|  balance|k_symbol|bank|account|
+--------+----------+------+------+---------+------+---------+--------+----+-------+
| 3569270|      1169|971031|PRIJEM|     null|206.50|114201.60|    UROK|null|   null|
| 3567456|      1120|971031|PRIJEM|     null| 94.80| 23446.00|    UROK|null|   null|
| 3566517|      1094|971031|PRIJEM|     null|109.80| 25796.50|    UROK|null|   null|
+--------+----------+------+------+---------+------+---------+--------+----+-------+
only showing top 3 rows



+-------+----------+------+------+--------+--------+------+
|loan_id|account_id|  date|amount|duration|payments|status|
+-------+----------+------+------+--------+--------+------+
|   5314|      1787|930705| 96396|      12| 8033.00|     B|
|   5316|      1801|930711|165960|      36| 4610.00|     A|
|   6863|      9188|930728|127080|      60| 2118.00|     A|
+-------+----------+------+------+--------+--------+------+
only showing top 3 rows



+----------+-----------+----------------+------+
|account_id|district_id|       frequency|  date|
+----------+-----------+----------------+------+
|       576|         55|POPLATEK MESICNE|930101|
|      3818|         74|POPLATEK MESICNE|930101|
|       704|         55|POPLATEK MESICNE|930101|
+----------+-----------+----------------+------+
only showing top 3 rows



+-------+-------+-------+---------------+
|card_id|disp_id|   type|         issued|
+-------+-------+-------+---------------+
|   1005|   9285|classic|931107 00:00:00|
|    104|    588|classic|940119 00:00:00|
|    747|   4915|classic|940205 00:00:00|
+-------+-------+-------+---------------+
only showing top 3 rows



+---------+------+------------+-----------+
|client_id|gender|birth_number|district_id|
+---------+------+------------+-----------+
|        1|Female|      706213|         18|
|        2|  Male|      450204|          1|
|        3|Female|      406009|          1|
+---------+------+------------+-----------+
only showing top 3 rows



+--------+----------+-------+----------+-------+--------+
|order_id|account_id|bank_to|account_to| amount|k_symbol|
+--------+----------+-------+----------+-------+--------+
|   29401|         1|     YZ|  87144583|2452.00|    SIPO|
|   29402|         2|     ST|  89597016|3372.70|    UVER|
|   29403|         2|     QR|  13943797|7266.00|    SIPO|
+--------+----------+-------+----------+-------+--------+
only showing top 3 rows



+-------+---------+----------+---------+
|disp_id|client_id|account_id|     type|
+-------+---------+----------+---------+
|      1|        1|         1|    OWNER|
|      2|        2|         2|    OWNER|
|      3|        3|         2|DISPONENT|
+-------+---------+----------+---------+
only showing top 3 rows



+---+-----------+---------------+-------+---+---+---+---+---+-----+-----+----+----+---+-----+-----+
| A1|         A2|             A3|     A4| A5| A6| A7| A8| A9|  A10|  A11| A12| A13|A14|  A15|  A16|
+---+-----------+---------------+-------+---+---+---+---+---+-----+-----+----+----+---+-----+-----+
|  1|Hl.m. Praha|         Prague|1204953|  0|  0|  0|  1|  1|100.0|12541|0.29|0.43|167|85677|99107|
|  2|    Benesov|central Bohemia|  88884| 80| 26|  6|  2|  5| 46.7| 8507|1.67|1.85|132| 2159| 2674|
|  3|     Beroun|central Bohemia|  75232| 55| 26|  4|  1|  5| 41.7| 8980|1.95|2.21|111| 2824| 2813|
+---+-----------+---------------+-------+---+---+---+---+---+-----+-----+----+----+---+-----+-----+
only showing top 3 rows



In [2]:
spark.sql(f"SHOW TABLES FROM {db_name}").show()

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|vr_testdb|            trans|      false|
|vr_testdb|             loan|      false|
|vr_testdb|          account|      false|
|vr_testdb|             card|      false|
|vr_testdb|           client|      false|
|vr_testdb|           orders|      false|
|vr_testdb|      disposition|      false|
|vr_testdb|         district|      false|
|vr_testdb|    trans_cleaned|      false|
|vr_testdb|loan_full_cleaned|      false|
|vr_testdb|    order_cleaned|      false|
|vr_testdb|   client_cleaned|      false|
+---------+-----------------+-----------+



In [3]:
spark.sql(f"select * from {db_name}.trans").show()

+--------+----------+------+------+-------------+-------+-------+--------+----+--------+
|trans_id|account_id|  date|  type|    operation| amount|balance|k_symbol|bank| account|
+--------+----------+------+------+-------------+-------+-------+--------+----+--------+
|  695247|      2378|930101|PRIJEM|        VKLAD| 700.00| 700.00|    null|null|    null|
|  171812|       576|930101|PRIJEM|        VKLAD| 900.00| 900.00|    null|null|    null|
|  207264|       704|930101|PRIJEM|        VKLAD|1000.00|1000.00|    null|null|    null|
| 1117247|      3818|930101|PRIJEM|        VKLAD| 600.00| 600.00|    null|null|    null|
|  579373|      1972|930102|PRIJEM|        VKLAD| 400.00| 400.00|    null|null|    null|
|  771035|      2632|930102|PRIJEM|        VKLAD|1100.00|1100.00|    null|null|    null|
|  452728|      1539|930103|PRIJEM|        VKLAD| 600.00| 600.00|    null|null|    null|
|  725751|      2484|930103|PRIJEM|        VKLAD|1100.00|1100.00|    null|null|    null|
|  497211|      1695|

In [4]:
spark.stop()